In [ ]:
import tkinter as tk
# filedialog is used in this case to save the file path selected by the user.
from tkinter import filedialog
#import pandas to handle the file.
import pandas as pd
import folium
import subprocess
import webbrowser
from http.server import BaseHTTPRequestHandler, HTTPServer


root = tk.Tk()
file_path = ""

def open_and_prep():
    # global is needed to interact with variables in the global name space
    global file_path
    # askopefilename is used to retrieve the file path and file name.
    file_path = filedialog.askopenfilename()
def choose_route():
    global route
    route = 'V'

def process_open_file():
    global file_path
    # do what you want with the file here.
    #Read the .csv
    history = pd.read_csv(file_path,encoding='latin1')
    #Sorting by 'Data'
    hy = history.sort_values(['Data'])
    #
    lat = hy['Lat'][:500].values
    long = hy['Long'][:500].values
    name = hy['Nome'].values
    data = hy['Data']
    cordinates = []

    mapa = folium.Map(location=[-20.3222,-40.3381],zoom_start=11)

    valid = False

    #Selecting id: Vitoria=V, Serra=S, Cariacica_262=C, Vila_Velha=VV

    rep = route
    id = {'V':'Vitoria', 'S':'Serra', 'C':'Cariacica_262','VV':'Vila_Velha'}

    for lati,longi,n in zip(lat,long,name):
        if n == id[rep]:
            cordinates.append([lati,longi])

    MP=folium.PolyLine(locations=cordinates,weight=2)

    for la,lo,nm,dt in zip(lat,long,name,data):
        if nm == id[rep]:
            folium.Marker([la, lo], popup=dt,tooltip=nm).add_to(mapa)
            mapa.add_child(MP)
    
    if file_path != "":
        # opens file from file path and prints each line.
        with open(file_path,"r") as testr:


        # ------------------------------------------------------------------------------------------------
        # so let's write a custom temporary-HTML renderer
        # pretty much copy-paste of this answer: https://stackoverflow.com/a/38945907/3494126

            PORT = 7000
            HOST = '127.0.0.199'
            SERVER_ADDRESS = '{host}:{port}'.format(host=HOST, port=PORT)
            FULL_SERVER_ADDRESS = 'http://' + SERVER_ADDRESS


            def TemproraryHttpServer(page_content_type, raw_data):
                """
                A simpe, temprorary http web server on the pure Python 3.
                It has features for processing pages with a XML or HTML content.
                """

                class HTTPServerRequestHandler(BaseHTTPRequestHandler):
                    """
                    An handler of request for the server, hosting XML-pages.
                    """

                    def do_GET(self):
                        """Handle GET requests"""

                        # response from page
                        self.send_response(200)

                        # set up headers for pages
                        content_type = 'text/{0}'.format(page_content_type)
                        self.send_header('Content-type', content_type)
                        self.end_headers()

                        # writing data on a page
                        self.wfile.write(bytes(raw_data, encoding='utf'))

                        return

                if page_content_type not in ['html', 'xml']:
                    raise ValueError('This server can serve only HTML or XML pages.')

                page_content_type = page_content_type

                # kill a process, hosted on a localhost:PORT
                subprocess.call(['fuser', '-k', '{0}/tcp'.format(PORT)])

                # Started creating a temprorary http server.
                httpd = HTTPServer((HOST, PORT), HTTPServerRequestHandler)

                # run a temprorary http server
                httpd.serve_forever()


            def run_html_server(html_data=None):

                if html_data is None:
                    html_data = """
                    <!DOCTYPE html>
                    <html>
                    <head>
                    <title>Mapa de Rotas</title>
                    </head>
                    <body>
                    <h1></h1>
                    <p></p>
                    </body>
                    </html>
                    """

                # open in a browser URL and see a result
                webbrowser.open(FULL_SERVER_ADDRESS)

                # run server
                TemproraryHttpServer('html', html_data)

            # ------------------------------------------------------------------------------------------------


            # now let's save the visualization into the temp file and render it

            from tempfile import NamedTemporaryFile
            tmp = NamedTemporaryFile()
            mapa.save(tmp.name)
            with open(tmp.name) as f:
                folium_map_html = f.read()

            run_html_server(folium_map_html)

            

# create Button that link to methods used to get file path.
tk.Button(root, text="Abrir arquivo", command=open_and_prep).pack()
# create Button to choose wich route that you want to see.
tk.Button(root, text="Escolher rota", command=choose_route).pack()
# create Button that link to methods used to process said file.
tk.Button(root, text="Printar Mapa", command=process_open_file).pack()

root.mainloop()

127.0.0.1 - - [11/Mar/2019 17:06:25] "GET / HTTP/1.1" 200 -


In [5]:
file_path

'/home/cobata/gh/Map_aplication-Python/location_and_signal_report1.csv'